In [105]:
import math
import json
import os
import random
from IPython import display
import sympy as sp
import numpy as np
import pandas as pd
import sklearn

pd.set_option('display.max_rows', None)

%run config.py

Utility functions

In [106]:
# Colors for terminal output
GREEN = "\033[92m"
RED = "\033[91m"
YELLOW = "\033[93m"
BLUE = "\033[94m"
CYAN = "\033[96m"
MAGENTA = "\033[95m"
BOLD = "\033[1m"
UNDERLINE = "\033[4m"
STANDARD = "\033[0m"

In [107]:
# Sequencer and Desequencer
def csv2sequence(csvFile,seqFile,pref):
	with open(csvFile, 'r') as f:
		lines = f.readlines()
		with open(seqFile, 'w') as g:
			for line in lines:
				for token in line.split(','):
					if token.startswith(pref):
						token = token[len(pref):]
					token = token.strip()
					if token!="" and token!='\n':
						g.write(token+'\n')

def sequence2csv(seqFile,csvFile,dataWidth,pref):
	with open(seqFile, 'r') as f:
		lines = f.readlines()
		dataW = dataWidth
		with open(csvFile, 'w') as g:
			for line in lines:
				g.write(pref+line.strip())
				dataW -= 1
				if dataW==0:
					g.write('\n')
					dataW = dataWidth
				else:
					g.write(',')

In [108]:
# Function to show the generated function
def showFunction():
	exec(open("generated.py").read())
	import IPython.display as disp
	disp.display(spExpr)
# showFunction()

In [109]:
# Function to show the generated BM
def showBM():
	display.Image("bondmachine.png")

environment creation functions

In [110]:
# Function create an environment with a random mathematical expression
def generateRandom():
	! echo "SOURCE_FLEXPY=generated.py" > source.mk
	! make clean > /dev/null
	! echo "SOURCE_FLEXPY=generated.py" > source.mk
	! flexpytester --generate -e symbols -s generated.py -i inputs.csv -o outputs.csv --prefix --config decayFactor=3 > /dev/null
	! bmhelper apply > /dev/null 
	! make show > /dev/null
	csv2sequence('inputs.csv','inputs.seq',"0f")
	csv2sequence('outputs.csv','outputs.seq',"0f")
# generateRandom()

In [111]:
# Function to generate the test cases of an existing mathematical expression
def generateExistent():
	! echo "SOURCE_FLEXPY=generated.py" > source.mk
	! make clean > /dev/null
	! echo "SOURCE_FLEXPY=generated.py" > source.mk
	! flexpytester --compute -e generated.py -i inputs.csv -o outputs.csv --prefix > /dev/null
	! bmhelper apply > /dev/null 
	! make show > /dev/null
	csv2sequence('inputs.csv','inputs.seq',"0f")
	csv2sequence('outputs.csv','outputs.seq',"0f")
# generateExistent()

In [112]:
# Function to generate the test cases of an existing single neuron within the library
def generateNeuron(library, neuron):
	! echo "SOURCE_BASM=out.basm" > source.mk
	! make clean > /dev/null
	! echo "SOURCE_BASM=out.basm" > source.mk
	! rm -f notok
	! fragtester -neuron-lib-path {library} -fragment-file {neuron}.basm -save-expression generated.py || touch notok > /dev/null
	if os.path.exists("notok"):
		return False
	! flexpytester --compute -e generated.py -i inputs.csv -o outputs.csv --prefix > /dev/null
	! bmhelper apply > /dev/null
	! make show > /dev/null
	csv2sequence('inputs.csv','inputs.seq',"0f")
	csv2sequence('outputs.csv','outputs.seq',"0f")
	! echo "{" > statistics.json
	! echo -n "\"" >> statistics.json
	! echo -n {neuron} >> statistics.json
	! echo "\": 1" >> statistics.json
	! echo "}" >> statistics.json
	return True
# generateNeuron("library", "addargfullargfull")

In [113]:
# Function to generate the test cases of an existing single neuron within the library
def placeHolderNeuron(library, neuron):
	! echo "{" > statistics.json
	! echo -n "\"" >> statistics.json
	! echo -n {neuron} >> statistics.json
	! echo "\": 0" >> statistics.json
	! echo "}" >> statistics.json

 Simulation beckends

In [114]:
# BondMachine internal simulator
def runSimbatch():
	! make simbatch > /dev/null
# runSimbatch()

In [115]:
# BondMachine hardware run
def runBMsim():
	! make deploycollect > /dev/null
	! mv working_dir/bmsim_outputs.seq bmsim_outputs.seq
	! bondmachine -bondmachine-file working_dir/bondmachine.json -list-outputs | wc -l > num_outputs.txt
	# Read the number of outputs
	with open('num_outputs.txt','r') as f:
		num_outputs = int(f.read())
	sequence2csv('bmsim_outputs.seq','bmsim_outputs.csv',num_outputs,"")
# runBMsim()

In [116]:
# HLS simulator
def rubHLSsim():
	print ("hlsim")
# runHLSsim()

Analysis

In [117]:
def analyzeSimbatch():
	# Load the target outputs from the output.csv file
	targetData = np.loadtxt('outputs.csv', delimiter=',')
	simbatchData = np.loadtxt('simbatch_outputs.csv', delimiter=',')
	simbatchMSE=sklearn.metrics.mean_squared_error(targetData, simbatchData)

	data = {
		"Dataset": ["Sympy target", "Simbatch"],
		"MSE" : [0.0, simbatchMSE]
	}
	df = pd.DataFrame(data)
	return df.style.hide(axis="index"), simbatchMSE

	

Statistics management functions

In [118]:
def plotHistogram(df):
	# Plot the histogram
	df['occurrences'].plot(kind='bar', figsize=(10, 6), color='skyblue')


In [119]:
def loadRun(mse):
	with open('statistics.json', 'r') as f:
		data = json.load(f)
	
	df = pd.DataFrame.from_dict(data, orient='index', columns=['occurrences'])
	totOccurrences = df['occurrences'].sum()
	newCol=mse
	if totOccurrences > 0:
		newCol = newCol/totOccurrences
		df['error'] = newCol
	else:
		df['error'] = 0.0
	return df

In [120]:
def PatchGlobalStats(global_stats, df):
	# If the global statistics DataFrame is empty, initialize it with the same rows as the current DataFrame
	for index, row in df.iterrows():
		if index not in global_stats.index:
			global_stats.loc[index] = [0.0, 0.0]

	# Sum the occurrences to the global statistics
	for index, row in df.iterrows():
		global_stats.at[index, 'occurrences'] += row['occurrences']
		global_stats.at[index, 'error'] += row['error']
	# Save the updated global statistics
	global_stats.to_csv('global_statistics')
	return global_stats


In [121]:
def loadGlobalStats():
	# Load the global statistics from the CSV file if it exists
	if os.path.exists('global_statistics'):
		global_stats = pd.read_csv('global_statistics', index_col=0)
	else:
		# Create an empty DataFrame with the same columns as the global statistics
		global_stats = pd.DataFrame(columns=['occurrences', 'error'])
	return global_stats

Interactive sessions

In [122]:
# Single run test, it executes the entire process and return a DataFrame (df) with the errors
if False:	
	# generateRandom()
	# generateExistent()
	generateNeuron("library", "addargfullargfull")
	runSimbatch()
	# runBMsim()
	# runHLSsim()
	errors,mse=analyzeSimbatch()
	# errors
	df=loadRun(mse)

In [123]:
# Load the historical global statistics and patch it with the new data (df)
if False:
	gs=loadGlobalStats()
	gs=PatchGlobalStats(gs, df)
	gs

In [ ]:
# Iterate al the neurons in the library and test them singly, patching the global statistics
for neuronFile in os.listdir("library"):
	if not neuronFile.endswith(".basm"):
		continue
	neuron = neuronFile.split(".")[0]
	# generateRandom()
	# generateExistent()
	if generateNeuron("library", neuron):
		runSimbatch()
		# runBMsim()
		# runHLSsim()
		errors,mse=analyzeSimbatch()
	else:
		placeHolderNeuron("library", neuron)

	df=loadRun(0.0)
	gs=loadGlobalStats()
	gs=PatchGlobalStats(gs, df)
gs

,occurrences,error
powargimagargimag,0.0,0.0
multargimagargfull,0.0,0.0
multargimagnumimag,0.0,0.0
powargimagnumfull,0.0,0.0
